<a href="https://colab.research.google.com/github/leticiafaria7/alura-courses/blob/main/apache-spark/manipulando_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Configuração do ambiente

In [1]:
# instalar as dependências

# pyspark (não pode ter espaço antes e depois do ==)
!pip install pyspark==3.3.1

# findspark - torna o pyspark uma biblioteca possível de ser importada regularmente
!pip install -q findspark

# instalar um kit de desenvolvimento java (JDK) que vai permitir rodar código na
# linguagem Scala e nas máquinas virtuais Java (JVM), que é como o spark foi construído
# VERIFICAR SE O ARQUIVO EXISTE NO SITE
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.4/spark-3.3.4-bin-hadoop3.tgz # baixar, através da ferramenta wget, os arquivos do Spark na máquina virtual do Google

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845494 sha256=0357942c8d9796ee181c6d51c82bfce5f52f8a449294e5e8f5b362ef687c0799
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# descompactar o arquivo que fizemos download

!tar xf spark-3.3.4-bin-hadoop3.tgz

In [3]:
# bibliotecas

import findspark
from pyspark.sql import SparkSession
from google.colab import drive
import zipfile
import pandas as pd

from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [4]:
# usar o findspark para permitir a importação dos pacotes necessários para utilizar o PySpark
findspark.init()

# criar a seção Spark
spark = SparkSession.builder\
  .master('local[*]')\
    .appName('Iniciando com Spark')\
      .config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
# montar o drive

drive.mount('/content/drive')

Mounted at /content/drive


# 1. Ler os dados

In [6]:
path_empresas = '/content/drive/MyDrive/5. Cursos/programming/alura-courses/apache-spark/dados/empresas'
empresas = spark.read.csv(path_empresas, sep = ';', inferSchema = True)

path_estabelecimentos = '/content/drive/MyDrive/5. Cursos/programming/alura-courses/apache-spark/dados/estabelecimentos'
estabelecimentos = spark.read.csv(path_estabelecimentos, sep = ';', inferSchema = True)

path_socios = '/content/drive/MyDrive/5. Cursos/programming/alura-courses/apache-spark/dados/socios'
socios = spark.read.csv(path_socios, sep = ';', inferSchema = True)

In [7]:
empresas.count()

4585679

# 2. Manipulando os dados

In [8]:
# operações básicas

empresas.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None


In [9]:
# renomear colunas

colunas_empresas = ['cnpj_basico', 'razao_social', 'natureza_juridica', 'qualificacao_responsavel', 'capital_social', 'porte', 'ente_federativo_responsavel']

colunas_estabelecimentos = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial',
                            'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral',
                            'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal',
                            'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento',
                            'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax',
                            'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

colunas_socios = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social',
                  'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade',
                  'pais', 'representante_legal', 'nome_do_representante',
                  'qualificacao_do_representante_legal', 'faixa_etaria']


for index, colName in enumerate(colunas_empresas):
  empresas = empresas.withColumnRenamed(f"_c{index}", colName)

for index, colName in enumerate(colunas_estabelecimentos):
  estabelecimentos = estabelecimentos.withColumnRenamed(f"_c{index}", colName)

for index, colName in enumerate(colunas_socios):
  socios = socios.withColumnRenamed(f"_c{index}", colName)

In [10]:
empresas.columns

['cnpj_basico',
 'razao_social',
 'natureza_juridica',
 'qualificacao_responsavel',
 'capital_social',
 'porte',
 'ente_federativo_responsavel']

In [11]:
# socios

socios.limit(5).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,19940725,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,19940725,NaN,***000000**,None,0,7
2,5813,2,CELSO EDUARDO DE CASTRO STEPHAN,***786068**,49,19940516,NaN,***000000**,None,0,8
3,5813,2,EDUARDO BERRINGER STEPHAN,***442348**,49,19940516,NaN,***000000**,None,0,5
4,14798,2,HANNE MAHFOUD FADEL,***760388**,49,19940609,NaN,***000000**,None,0,8


In [12]:
# estabelecimentos

estabelecimentos.limit(5).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,20011029,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
1,2818,1,43,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
2,3110,1,7,1,None,8,19971231,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
3,3733,1,80,1,None,8,20081231,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,NaN
4,4628,3,27,2,EMBROIDERY & GIFT,8,19980429,1,None,NaN,...,7075,None,None,None,None,NaN,None,None,None,NaN


In [13]:
# ver tipos das variáveis

empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



In [14]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: integer (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



In [15]:
estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: integer (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: str

In [16]:
# alterar o tipo da coluna de capital social da empresa

empresas = empresas.withColumn('capital_social', f.regexp_replace('capital_social', ',', '.'))
empresas.limit(5).toPandas()

,cnpj_basico,razao_social,natureza_juridica,qualificacao_responsavel,capital_social,porte,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.00,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.00,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.00,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.00,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.00,1,None


In [17]:
empresas = empresas.withColumn('capital_social', f.col('capital_social').cast(DoubleType()))
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: double (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



## Converter string para data

In [18]:
df = spark.createDataFrame([(20200924,), (20201022,), (20210215,)], ['data'])
df.toPandas()

,data
0,20200924
1,20201022
2,20210215


In [19]:
df.printSchema()

root
 |-- data: long (nullable = true)



In [20]:
df = df.withColumn('data', f.to_date(df.data.cast(StringType()), 'yyyyMMdd'))
df.printSchema()

root
 |-- data: date (nullable = true)



In [21]:
estabelecimentos = estabelecimentos\
.withColumn(
    'data_situacao_cadastral',
    f.to_date(estabelecimentos.data_situacao_cadastral.cast(StringType()), 'yyyyMMdd')
)\
.withColumn(
    'data_de_inicio_atividade',
    f.to_date(estabelecimentos.data_de_inicio_atividade.cast(StringType()), 'yyyyMMdd')
)\
.withColumn(
    'data_da_situacao_especial',
    f.to_date(estabelecimentos.data_da_situacao_especial.cast(StringType()), 'yyyyMMdd')
)

estabelecimentos.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [22]:
estabelecimentos.limit(2).toPandas()

,cnpj_basico,cnpj_ordem,cnpj_dv,identificador_matriz_filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_da_cidade_no_exterior,pais,...,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_do_fax,fax,correio_eletronico,situacao_especial,data_da_situacao_especial
0,1879,1,96,1,PIRAMIDE M. C.,8,2001-10-29,1,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None
1,2818,1,43,1,None,8,2008-12-31,71,None,NaN,...,7107,None,None,None,None,NaN,None,None,None,None


In [23]:
sorted(socios.columns)

['cnpj_basico',
 'cnpj_ou_cpf_do_socio',
 'data_de_entrada_sociedade',
 'faixa_etaria',
 'identificador_de_socio',
 'nome_do_representante',
 'nome_do_socio_ou_razao_social',
 'pais',
 'qualificacao_do_representante_legal',
 'qualificacao_do_socio',
 'representante_legal']

In [24]:
socios = socios\
.withColumn(
    'data_de_entrada_sociedade',
    f.to_date(f.col('data_de_entrada_sociedade').cast(StringType()), 'yyyyMMdd')
)

socios.limit(2).toPandas()

,cnpj_basico,identificador_de_socio,nome_do_socio_ou_razao_social,cnpj_ou_cpf_do_socio,qualificacao_do_socio,data_de_entrada_sociedade,pais,representante_legal,nome_do_representante,qualificacao_do_representante_legal,faixa_etaria
0,411,2,LILIANA PATRICIA GUASTAVINO,***678188**,22,1994-07-25,NaN,***000000**,None,0,7
1,411,2,CRISTINA HUNDERTMARK,***637848**,28,1994-07-25,NaN,***000000**,None,0,7


# Seleções e consultas

In [25]:
# selecionar informações

empresas\
.select('*')\
.show(5, truncate = False) # False = mostrar a informação completa

+-----------+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------------------+
|cnpj_basico|razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo_responsavel|
+-----------+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------------------+
|306        |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0.0           |1    |null                       |
|1355       |BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0.0           |5    |null                       |
|4820

In [26]:
empresas.columns

['cnpj_basico',
 'razao_social',
 'natureza_juridica',
 'qualificacao_responsavel',
 'capital_social',
 'porte',
 'ente_federativo_responsavel']

In [27]:
empresas\
.select('natureza_juridica', 'porte', 'capital_social')\
.show(5)

+-----------------+-----+--------------+
|natureza_juridica|porte|capital_social|
+-----------------+-----+--------------+
|             2240|    1|           0.0|
|             2062|    5|           0.0|
|             3034|    5|           0.0|
|             2135|    5|           0.0|
|             2062|    1|        4000.0|
+-----------------+-----+--------------+
only showing top 5 rows



In [28]:
socios\
.select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
.show(5, False)

+-------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social  |faixa_etaria|ano_de_entrada|
+-------------------------------+------------+--------------+
|LILIANA PATRICIA GUASTAVINO    |7           |1994          |
|CRISTINA HUNDERTMARK           |7           |1994          |
|CELSO EDUARDO DE CASTRO STEPHAN|8           |1994          |
|EDUARDO BERRINGER STEPHAN      |5           |1994          |
|HANNE MAHFOUD FADEL            |8           |1994          |
+-------------------------------+------------+--------------+
only showing top 5 rows



In [29]:
estabelecimentos.columns

['cnpj_basico',
 'cnpj_ordem',
 'cnpj_dv',
 'identificador_matriz_filial',
 'nome_fantasia',
 'situacao_cadastral',
 'data_situacao_cadastral',
 'motivo_situacao_cadastral',
 'nome_da_cidade_no_exterior',
 'pais',
 'data_de_inicio_atividade',
 'cnae_fiscal_principal',
 'cnae_fiscal_secundaria',
 'tipo_de_logradouro',
 'logradouro',
 'numero',
 'complemento',
 'bairro',
 'cep',
 'uf',
 'municipio',
 'ddd_1',
 'telefone_1',
 'ddd_2',
 'telefone_2',
 'ddd_do_fax',
 'fax',
 'correio_eletronico',
 'situacao_especial',
 'data_da_situacao_especial']

In [30]:
estabelecimentos\
.select('nome_fantasia', 'municipio', f.year('data_de_inicio_atividade').alias('ano_inicio_atividade'), f.month('data_de_inicio_atividade').alias('mes_inicio_atividade'))\
.show(5, False)

+-----------------+---------+--------------------+--------------------+
|nome_fantasia    |municipio|ano_inicio_atividade|mes_inicio_atividade|
+-----------------+---------+--------------------+--------------------+
|PIRAMIDE M. C.   |7107     |1994                |5                   |
|null             |7107     |1994                |5                   |
|null             |7107     |1994                |5                   |
|null             |7107     |1994                |5                   |
|EMBROIDERY & GIFT|7075     |1995                |5                   |
+-----------------+---------+--------------------+--------------------+
only showing top 5 rows



# Identificando valores nulos

In [31]:
df = spark.createDataFrame([(1,), (2,), (3,), (None,)], ['data'])
df.toPandas()

,data
0,1.0
1,2.0
2,3.0
3,NaN


In [32]:
df.show()

+----+
|data|
+----+
|   1|
|   2|
|   3|
|null|
+----+



In [33]:
df = spark.createDataFrame([(1.,), (2.,), (3.,), (float('nan'),)], ['data'])
df.toPandas()

,data
0,1.0
1,2.0
2,3.0
3,NaN


In [34]:
df.show()

+----+
|data|
+----+
| 1.0|
| 2.0|
| 3.0|
| NaN|
+----+



In [35]:
df = spark.createDataFrame([('1',), ('2',), ('3',), (None,)], ['data'])
df.toPandas()

,data
0,1
1,2
2,3
3,None


In [36]:
df.show()

+----+
|data|
+----+
|   1|
|   2|
|   3|
|null|
+----+



In [37]:
# contar nulos por coluna

socios.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in socios.columns]).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+-------+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|   pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+-------+-------------------+---------------------+-----------------------------------+------------+
|          0|                     0|                          208|                1234|                    0|                        1|2038255|                  0|              1995432|                                  0|           0|
+-----------+----------------------+------------------------

In [38]:
# substitui os nulos do pais mas não os nulos do nome do representante
socios.select('pais', 'nome_do_representante').na.fill(0).limit(5).toPandas()

,pais,nome_do_representante
0,0,None
1,0,None
2,0,None
3,0,None
4,0,None


In [39]:
socios.select('pais', 'nome_do_representante').na.fill('-').limit(5).toPandas()

,pais,nome_do_representante
0,NaN,-
1,NaN,-
2,NaN,-
3,NaN,-
4,NaN,-


# Ordenando os dados

In [40]:
socios\
.select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
.orderBy('ano_de_entrada', ascending = False)\
.show(5, False)

+-----------------------------+------------+--------------+
|nome_do_socio_ou_razao_social|faixa_etaria|ano_de_entrada|
+-----------------------------+------------+--------------+
|JOSE HUMBERTO PAIVA          |6           |2021          |
|BENILDES BARBOSA RODRIGUES   |8           |2021          |
|MARCELO MOCELIN              |5           |2021          |
|ROBERTA BENELLI              |4           |2021          |
|EDUARDO DE ANDRADE PEIXOTO   |5           |2021          |
+-----------------------------+------------+--------------+
only showing top 5 rows



In [41]:
socios\
.select('nome_do_socio_ou_razao_social', 'faixa_etaria', f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
.orderBy(['ano_de_entrada', 'faixa_etaria'], ascending = [False, False])\
.show(10, False)

+---------------------------------+------------+--------------+
|nome_do_socio_ou_razao_social    |faixa_etaria|ano_de_entrada|
+---------------------------------+------------+--------------+
|ANTONIO TAVARES DE ANDRADE       |9           |2021          |
|ANNA MARIA TELLES FERREIRA SANTOS|9           |2021          |
|ANTONIA DE SOUSA VIEIRA          |9           |2021          |
|AURA MARIA DE ANDRADE            |9           |2021          |
|SONIA MARQUES SAMAJA             |9           |2021          |
|CARLOS ERANE DE AGUIAR           |9           |2021          |
|MATILDE CONCEICAO DE JESUS       |9           |2021          |
|MANUEL TAVARES DE SOUSA          |9           |2021          |
|ALBERTO DE BANDOS MENDES         |9           |2021          |
|EMERSON AZEVEDO                  |9           |2021          |
+---------------------------------+------------+--------------+
only showing top 10 rows



# Filtrar os dados

In [42]:
empresas\
.where('capital_social==50')\
.show(5, False)

+-----------+------------------------------------+-----------------+------------------------+--------------+-----+---------------------------+
|cnpj_basico|razao_social                        |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo_responsavel|
+-----------+------------------------------------+-----------------+------------------------+--------------+-----+---------------------------+
|17350147   |ERIK MARCELO DOS SANTOS 42107848858 |2135             |50                      |50.0          |1    |null                       |
|17833214   |ALEXANDRE MACHADO LIMA 73750123772  |2135             |50                      |50.0          |1    |null                       |
|20860830   |YASMIN MOURA DA FONSECA 13457709793 |2135             |50                      |50.0          |1    |null                       |
|22242856   |JOAO CESAR MESSIAS 08707149883      |2135             |50                      |50.0          |1    |null                       |

In [43]:
socios.count()

2046430

In [44]:
socios\
.select('nome_do_socio_ou_razao_social')\
.filter(socios.nome_do_socio_ou_razao_social.startswith('LETICIA'))\
.filter(socios.nome_do_socio_ou_razao_social.endswith('FARIA'))\
.limit(10)\
.toPandas()

,nome_do_socio_ou_razao_social
0,LETICIA MARIA TELO DE FARIA
1,LETICIA RAPHAELA ROSA DE FARIA
2,LETICIA APARECIDA DE FARIA
3,LETICIA DE FARIA
4,LETICIA DA SILVA FARIA


# Comando LIKE

In [45]:
df = spark.createDataFrame([('RESTAURANTE DO RUI',), ('Juca restaurantes ltda',), ('Joca Restaurante',)], ['data'])
df.toPandas()

,data
0,RESTAURANTE DO RUI
1,Juca restaurantes ltda
2,Joca Restaurante


In [46]:
 # percentuais = qualquer posição dentro de uma string

df\
.where(f.upper(df.data).like('%RESTAURANTE%'))\
.show(truncate = False)

+----------------------+
|data                  |
+----------------------+
|RESTAURANTE DO RUI    |
|Juca restaurantes ltda|
|Joca Restaurante      |
+----------------------+



In [47]:
# remover % do inicio = começa com restaurante

df\
.where(f.upper(df.data).like('RESTAURANTE%'))\
.show(truncate = False)

+------------------+
|data              |
+------------------+
|RESTAURANTE DO RUI|
+------------------+



In [48]:
empresas.columns

['cnpj_basico',
 'razao_social',
 'natureza_juridica',
 'qualificacao_responsavel',
 'capital_social',
 'porte',
 'ente_federativo_responsavel']

In [49]:
empresas\
.select('razao_social', 'natureza_juridica', 'porte', 'capital_social')\
.filter(f.upper(empresas['razao_social']).like('%RESTAURANTE%'))\
.show(15, False)

+-------------------------------------------------------+-----------------+-----+--------------+
|razao_social                                           |natureza_juridica|porte|capital_social|
+-------------------------------------------------------+-----------------+-----+--------------+
|RESTAURANTE IMIGRANTE PORTUGUES LTDA.                  |2062             |5    |0.0           |
|MORAIS & CARVALHO RESTAURANTE E PIZZARIA LTDA          |2062             |1    |0.0           |
|BAR E RESTAURANTE PAGANOTTO LTDA                       |2062             |5    |0.0           |
|RODRIGUES & RODRIGUES RESTAURANTE LTDA                 |2062             |5    |0.0           |
|TEXAS RANCH BAR RESTAURANTE PRODUCOES ARTISTICAS E CULT|2062             |1    |0.0           |
|V V SANTOS RESTAURANTE BAR E ATIV DESPORTIVAS LTDA     |2062             |1    |0.0           |
|BAR E RESTAURANTE CASA DA QUINTA LTDA                  |2062             |1    |5000.0        |
|DON MUGO RESTAURANTE LTDA    

# Sumarizar os dados

In [50]:
socios\
.select(f.year('data_de_entrada_sociedade').alias('ano_de_entrada'))\
.where('ano_de_entrada >= 2010')\
.groupBy('ano_de_entrada')\
.count()\
.orderBy('ano_de_entrada', ascending = True)\
.show()

+--------------+------+
|ano_de_entrada| count|
+--------------+------+
|          2010| 79337|
|          2011| 83906|
|          2012| 80101|
|          2013| 83919|
|          2014| 80590|
|          2015| 80906|
|          2016| 81587|
|          2017| 90221|
|          2018| 99935|
|          2019|118248|
|          2020|125927|
|          2021| 56316|
+--------------+------+



In [51]:
empresas.columns

['cnpj_basico',
 'razao_social',
 'natureza_juridica',
 'qualificacao_responsavel',
 'capital_social',
 'porte',
 'ente_federativo_responsavel']

In [52]:
empresas\
.select('cnpj_basico', 'porte', 'capital_social')\
.groupBy('porte')\
.agg(
    f.avg('capital_social').alias('capital_social_medio'),
    f.count('cnpj_basico').alias('frequencia')
)\
.orderBy('porte', ascending = True)\
.show()

+-----+--------------------+----------+
|porte|capital_social_medio|frequencia|
+-----+--------------------+----------+
| null|    8.35421888053467|      5985|
|    1|  339994.53313506936|   3129043|
|    3|  2601001.7677092673|    115151|
|    5|   708660.4208249798|   1335500|
+-----+--------------------+----------+



In [53]:
empresas\
.select('capital_social')\
.summary()\
.show()

+-------+--------------------+
|summary|      capital_social|
+-------+--------------------+
|  count|             4585679|
|   mean|   503694.5478542675|
| stddev|2.1118691490537405E8|
|    min|                 0.0|
|    25%|                 0.0|
|    50%|              1000.0|
|    75%|              7000.0|
|    max|    3.22014670262E11|
+-------+--------------------+



# Juntar dataframes

In [54]:
produtos = spark.createDataFrame(
    [
        ('1', 'Bebidas', 'Água mineral'),
        ('2', 'Limpeza', 'Sabão em pó'),
        ('3', 'Frios', 'Queijo'),
        ('4', 'Bebidas', 'Refrigerante'),
        ('5', 'Pet', 'Ração para cães'),
    ],
    ['id', 'cat', 'prod']
)

impostos = spark.createDataFrame(
    [
        ('Bebidas', 0.15),
        ('Limpeza', 0.05),
        ('Frios', 0.065),
        ('Carnes', 0.08),
    ],
    ['cat', 'tax']
)

In [55]:
produtos.toPandas()

,id,cat,prod
0,1,Bebidas,Água mineral
1,2,Limpeza,Sabão em pó
2,3,Frios,Queijo
3,4,Bebidas,Refrigerante
4,5,Pet,Ração para cães


In [56]:
impostos.toPandas()

,cat,tax
0,Bebidas,0.150
1,Limpeza,0.050
2,Frios,0.065
3,Carnes,0.080


In [57]:
produtos.join(impostos, 'cat', how = 'inner')\
.sort('id')\
.show()

+-------+---+------------+-----+
|    cat| id|        prod|  tax|
+-------+---+------------+-----+
|Bebidas|  1|Água mineral| 0.15|
|Limpeza|  2| Sabão em pó| 0.05|
|  Frios|  3|      Queijo|0.065|
|Bebidas|  4|Refrigerante| 0.15|
+-------+---+------------+-----+



In [58]:
produtos.join(impostos, 'cat', how = 'left')\
.sort('id')\
.show()

+-------+---+---------------+-----+
|    cat| id|           prod|  tax|
+-------+---+---------------+-----+
|Bebidas|  1|   Água mineral| 0.15|
|Limpeza|  2|    Sabão em pó| 0.05|
|  Frios|  3|         Queijo|0.065|
|Bebidas|  4|   Refrigerante| 0.15|
|    Pet|  5|Ração para cães| null|
+-------+---+---------------+-----+



In [59]:
produtos.join(impostos, 'cat', how = 'right')\
.sort('id')\
.show()

+-------+----+------------+-----+
|    cat|  id|        prod|  tax|
+-------+----+------------+-----+
| Carnes|null|        null| 0.08|
|Bebidas|   1|Água mineral| 0.15|
|Limpeza|   2| Sabão em pó| 0.05|
|  Frios|   3|      Queijo|0.065|
|Bebidas|   4|Refrigerante| 0.15|
+-------+----+------------+-----+



In [60]:
produtos.join(impostos, 'cat', how = 'outer')\
.sort('id')\
.show()

+-------+----+---------------+-----+
|    cat|  id|           prod|  tax|
+-------+----+---------------+-----+
| Carnes|null|           null| 0.08|
|Bebidas|   1|   Água mineral| 0.15|
|Limpeza|   2|    Sabão em pó| 0.05|
|  Frios|   3|         Queijo|0.065|
|Bebidas|   4|   Refrigerante| 0.15|
|    Pet|   5|Ração para cães| null|
+-------+----+---------------+-----+



In [61]:
empresas_join = estabelecimentos.join(empresas, 'cnpj_basico', how = 'inner')

In [62]:
empresas_join.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_de_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_de_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (n

In [63]:
freq = empresas_join\
.select(
    'cnpj_basico',
    f.year('data_de_inicio_atividade').alias('data_de_inicio')
)\
.where('data_de_inicio >= 2010')\
.groupBy('data_de_inicio')\
.agg(f.count('cnpj_basico').alias('frequencia'))\
.orderBy('data_de_inicio', ascending = True)

In [64]:
freq.toPandas()

,data_de_inicio,frequencia
0,2010,154159
1,2011,172677
2,2012,232480
3,2013,198424
4,2014,202276
5,2015,212523
6,2016,265417
7,2017,237292
8,2018,275435
9,2019,325922


In [65]:
freq.union(
    freq.select(
        f.lit('Total').alias('data_de_inicio'),
        f.sum(freq.frequencia).alias('frequencia')
    )
).show()

+--------------+----------+
|data_de_inicio|frequencia|
+--------------+----------+
|          2010|    154159|
|          2011|    172677|
|          2012|    232480|
|          2013|    198424|
|          2014|    202276|
|          2015|    212523|
|          2016|    265417|
|          2017|    237292|
|          2018|    275435|
|          2019|    325922|
|          2020|    400654|
|          2021|    153275|
|         Total|   2830534|
+--------------+----------+



# Spark SQL

In [66]:
empresas.createOrReplaceTempView('empresasView')

In [67]:
spark.sql("SELECT * FROM empresasView").show(5)

+-----------+--------------------+-----------------+------------------------+--------------+-----+---------------------------+
|cnpj_basico|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo_responsavel|
+-----------+--------------------+-----------------+------------------------+--------------+-----+---------------------------+
|        306|FRANCAMAR REFRIGE...|             2240|                      49|           0.0|    1|                       null|
|       1355|BRASILEIRO & OLIV...|             2062|                      49|           0.0|    5|                       null|
|       4820|REGISTRO DE IMOVE...|             3034|                      32|           0.0|    5|                       null|
|       5347|ROSELY APARECIDA ...|             2135|                      50|           0.0|    5|                       null|
|       6846|BADU E FILHOS TEC...|             2062|                      49|        4000.0|    1|             

In [68]:
spark.sql("""
SELECT *
FROM empresasView
WHERE capital_social = 50
""")\
.show(5)

+-----------+--------------------+-----------------+------------------------+--------------+-----+---------------------------+
|cnpj_basico|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo_responsavel|
+-----------+--------------------+-----------------+------------------------+--------------+-----+---------------------------+
|   17350147|ERIK MARCELO DOS ...|             2135|                      50|          50.0|    1|                       null|
|   17833214|ALEXANDRE MACHADO...|             2135|                      50|          50.0|    1|                       null|
|   20860830|YASMIN MOURA DA F...|             2135|                      50|          50.0|    1|                       null|
|   22242856|JOAO CESAR MESSIA...|             2135|                      50|          50.0|    1|                       null|
|   23238540|EVERTON ROBERTO D...|             2135|                      50|          50.0|    1|             

In [71]:
spark.sql("""
SELECT porte, MEAN(capital_social) AS media
FROM empresasView
GROUP BY porte
""")\
.show(5)

+-----+------------------+
|porte|             media|
+-----+------------------+
| null|  8.35421888053467|
|    1|339994.53313506936|
|    3|2601001.7677092673|
|    5| 708660.4208249798|
+-----+------------------+



In [72]:
empresas_join.createOrReplaceTempView('empresasJoinView')

In [74]:
freq = spark\
.sql("""
SELECT YEAR(data_de_inicio_atividade) AS data_de_inicio, COUNT(cnpj_basico) AS count
FROM empresasJoinView
WHERE YEAR(data_de_inicio_atividade) >= 2010
GROUP BY data_de_inicio
ORDER BY data_de_inicio
""")

freq.show()

+--------------+------+
|data_de_inicio| count|
+--------------+------+
|          2010|154159|
|          2011|172677|
|          2012|232480|
|          2013|198424|
|          2014|202276|
|          2015|212523|
|          2016|265417|
|          2017|237292|
|          2018|275435|
|          2019|325922|
|          2020|400654|
|          2021|153275|
+--------------+------+



In [75]:
freq.createOrReplaceTempView('freqView')

spark\
.sql("""
SELECT *
FROM freqView
UNION ALL
SELECT 'Total' AS data_de_inicio, SUM(count) AS count
FROM freqView
""")\
.show()

+--------------+-------+
|data_de_inicio|  count|
+--------------+-------+
|          2010| 154159|
|          2011| 172677|
|          2012| 232480|
|          2013| 198424|
|          2014| 202276|
|          2015| 212523|
|          2016| 265417|
|          2017| 237292|
|          2018| 275435|
|          2019| 325922|
|          2020| 400654|
|          2021| 153275|
|         Total|2830534|
+--------------+-------+

